In [4]:
import re
from time import sleep
import itertools
import requests
from pathlib import Path
from bs4 import BeautifulSoup

URLS = {
    "hist_data":"https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/",
    "recent_data":"https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/",
}

for category, url in URLS.items():
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
   
    links = soup.find_all("a", href = re.compile(".pdf$|.txt$|.zip$"))

    for link in links:
        path = Path("./data/") / category 
        path.mkdir(parents=True, exist_ok=True)
        file_path = path / link["href"]
        mode = "w+b" if "pdf" or "zip" in link["href"] else "w+"
        file_url = url + link["href"]
        if not file_path.is_file():
            with requests.get(file_url) as r:
                with open(str(file_path), mode) as f:
                    f.write(r.content)
                    sleep(0.5)

            

In [59]:
from zipfile import ZipFile

with ZipFile('./data/recent_data/tageswerte_KL_00011_akt.zip', 'r') as zip:
    zip.printdir()


[<a href="BESCHREIBUNG_obsgermany_climate_daily_kl_recent_de.pdf">BESCHREIBUNG_obsgermany_climate_daily_kl_recent..&gt;</a>, <a href="DESCRIPTION_obsgermany_climate_daily_kl_recent_en.pdf">DESCRIPTION_obsgermany_climate_daily_kl_recent_..&gt;</a>]
